In [1]:
import pandas as pd
import numpy as np

# Upon Analysing the `vehicles.csv` it was observed that some good percent of data for missing for some fields, as shown below.
```
manufacturer     4.133714
model            1.236179
condition       40.785232
cylinders       41.622470
fuel             0.705819
transmission     0.598763
drive           30.586347
size            71.767476
type            21.752717
```
<p><b>
Upon checking the VIN number of some (5-10) from different category, it was evident the data is not completely made up and data matched to what was available. Thus all 425K records whose VIN was given, we did pull in additional information to compliment the existing data and fill in missing gaps.

To do so, an ETL job was spun in which 10 worker threads in Google Cloud fetched the information. It took 28 minutes to fetch data for all records. Below is sumamry of the data. 
</b></p>

In [2]:
df_enrich = pd.read_csv("data/enriched.csv")
df = pd.read_csv("data/vehicles.csv")

In [3]:
df_enrich = df_enrich[~df_enrich.duplicated()]
df_enrich.shape

(113786, 8)

In [4]:
df.shape

(426880, 18)

In [5]:
df.drop(columns=['id'], inplace=True)
df = df[~df.duplicated()]

In [6]:
df.shape

(370678, 17)

In [7]:
df_enrich.head()

,VIN_,BodyClass,Doors,DriveType,EngineCylinders,FuelTypePrimary,TransmissionStyle,Make
0,2C3CDZFJ5GH192418,coupe,2.0,rwd,8.0,Gasoline,NaN,dodge
1,2C3CDZBT2GH316605,coupe,2.0,rwd,8.0,Gasoline,NaN,dodge
2,KNDJ23AU7L7023949,sport utility vehicle (suv),4.0,4x2,4.0,Gasoline,continuously variable transmission (cvt),kia
3,KNDJN2A20J7579047,sport utility vehicle (suv),4.0,4x2,4.0,Gasoline,NaN,kia
4,19UDE2F32HA011136,sedan,4.0,4x2,4.0,Gasoline,dual-clutch transmission (dct),acura


In [8]:
df_merge = pd.merge(left=df, right=df_enrich, left_on='VIN', right_on='VIN_', how="left")

In [9]:
df_merge.shape

(370678, 25)

# Enrich Drive Type

In [10]:
df_enrich.DriveType.unique().tolist()

['rwd', '4x2', nan, 'awd', '4wd', 'fwd', '6x4', '2wd', '8x4', 'other', '6x6']

In [11]:
df.drive.unique().tolist()

[nan, 'rwd', '4wd', 'fwd']

In [12]:
df_enrich = df_enrich.replace({'DriveType': {'2wd': 'fwd', "awd":'4wd'}})   #

In [13]:
df_merge[(df_merge.drive.isna()) & (df_merge.DriveType.isin(['rwd', '4wd', 'fwd']))]

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,...,paint_color,state,VIN_,BodyClass,Doors,DriveType,EngineCylinders,FuelTypePrimary,TransmissionStyle,Make
19,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,57923.0,clean,...,white,al,3GTP1VEC4EG551563,pickup,4.0,rwd,8.0,Gasoline,NaN,gmc
50,auburn,25590,2015.0,gmc,sierra 1500 regular cab,good,6 cylinders,other,35290.0,clean,...,white,al,1GTN1TEH9FZ243281,pickup,2.0,rwd,6.0,Gasoline,NaN,gmc
58,auburn,32990,2019.0,chevrolet,silverado 1500 ld,good,NaN,other,6897.0,clean,...,black,al,2GCVKNEC8K1187169,pickup,NaN,4wd,8.0,Gasoline,NaN,chevrolet
74,auburn,25990,2015.0,lexus,nx 300h sport utility 4d,good,NaN,other,55783.0,clean,...,white,al,JTJBJRBZ8F2020836,sport utility vehicle (suv),5.0,4wd,4.0,Gasoline,NaN,lexus
84,auburn,19990,2015.0,infiniti,qx60 3.5 sport utility,good,NaN,other,96003.0,clean,...,silver,al,5N1AL0MM5FC519184,crossover utility vehicle (cuv),4.0,4wd,6.0,Gasoline,NaN,infiniti
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370569,wyoming,31590,2016.0,lexus,rx 350 sport utility 4d,good,6 cylinders,other,27139.0,clean,...,silver,wy,2T2BZMCA7GC014516,sport utility vehicle (suv),5.0,4wd,6.0,Gasoline,NaN,lexus
370570,wyoming,23590,2017.0,acura,rdx sport utility 4d,good,6 cylinders,gas,55971.0,clean,...,blue,wy,5J8TB4H79HL034983,sport utility vehicle (suv),5.0,4wd,6.0,Gasoline,automatic,acura
370655,wyoming,7477,2012.0,fiat,500,NaN,NaN,gas,64267.0,clean,...,NaN,wy,3C3CFFBRXCT171831,hatchback,2.0,fwd,4.0,Gasoline,NaN,fiat
370660,wyoming,30990,2019.0,alfa-romeo,romeo giulia ti sedan 4d,good,NaN,other,3883.0,clean,...,blue,wy,ZARFAMBN5K7621550,sedan,4.0,rwd,4.0,Gasoline,automatic,alfa romeo


In [14]:
df_merge.loc[(df_merge.drive.isna()) & (df_merge.DriveType.isin(['rwd', '4wd', 'fwd'])), 'drive'] = df_merge['DriveType']

In [15]:
df_merge

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,...,paint_color,state,VIN_,BodyClass,Doors,DriveType,EngineCylinders,FuelTypePrimary,TransmissionStyle,Make
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,az,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,ar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,fl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,ma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,nc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370673,wyoming,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,...,NaN,wy,1N4AA6AV6KC367801,sedan,4.0,4x2,NaN,Gasoline,NaN,nissan
370674,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,...,red,wy,7JR102FKXLG042696,sedan,4.0,NaN,4.0,Gasoline,automatic,volvo
370675,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,...,white,wy,1GYFZFR46LF088296,sport utility vehicle (suv),NaN,awd,4.0,Gasoline,automatic,cadillac
370676,wyoming,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,...,silver,wy,58ABK1GG4JU103853,sedan,4.0,4x2,6.0,Gasoline,automatic,lexus


- updated 14416 records

# Enrich Size

In [16]:
_type_lst = df_merge.type.unique()
_type_lst = [x for x in _type_lst.tolist() if x!=np.nan]
_type_lst

[nan,
 'pickup',
 'truck',
 'other',
 'coupe',
 'SUV',
 'hatchback',
 'mini-van',
 'sedan',
 'offroad',
 'bus',
 'van',
 'convertible',
 'wagon']

In [17]:
df_merge.BodyClass.unique()

array([nan, 'pickup', 'convertible', 'sport utility vehicle (suv)',
       'coupe', 'sedan', 'crossover utility vehicle (cuv)', 'hatchback',
       'cargo van', 'minivan', 'wagon',
       'incomplete - chassis cab (double cab)',
       'incomplete - chassis cab (single cab)', 'incomplete - cutaway',
       'incomplete - stripped chassis', 'van',
       'incomplete - chassis cab (number of cab unknown)', 'truck',
       'bus - school bus', 'incomplete',
       'incomplete - commercial chassis', 'roadster', 'step van ',
       'sport utility truck (sut)', 'motorcycle - cruiser',
       'motorcycle - sport', 'truck-tractor', 'motorcycle - touring ',
       'incomplete - motor home chassis',
       'incomplete - commercial bus chassis', 'low speed vehicle (lsv) ',
       'motorcycle - custom', 'motorcycle - street',
       'motorcycle - three-wheeled motorcycle (2 rear wheels)',
       'incomplete - school bus chassis', 'limousine',
       'motorcycle - standard',
       'motorcycle - unen

In [18]:
df_merge = df_merge.replace({'BodyClass': 
                             {'sport utility vehicle (suv)': 'SUV', 
                              'minivan':'mini-van',
                              'cargo van':'van',
                              'bus - school bus': 'bus'
                             },
                            })

In [19]:
df_merge[(df_merge.type.isna()) & df_merge.BodyClass.isin(_type_lst)]

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,...,paint_color,state,VIN_,BodyClass,Doors,DriveType,EngineCylinders,FuelTypePrimary,TransmissionStyle,Make
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,az,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,ar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,fl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,ma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,nc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370644,wyoming,65999,2017.0,NaN,2017,NaN,NaN,diesel,73000.0,clean,...,NaN,wy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370645,wyoming,16999,2014.0,NaN,2014,NaN,NaN,gas,89000.0,clean,...,NaN,wy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370646,wyoming,22999,2013.0,NaN,2013,NaN,NaN,gas,120000.0,clean,...,NaN,wy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370647,wyoming,21999,2017.0,NaN,2017,NaN,NaN,gas,51000.0,clean,...,NaN,wy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_merge.loc[(df_merge.type.isna()) & df_merge.BodyClass.isin(_type_lst), 'type'] = df_merge['BodyClass']

In [21]:
df_merge[df_merge.VIN.isin(['1FTSX21P05EB88346','1N6AD0EV8GN781910','3C63R3CL1FG678469'])]

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,...,paint_color,state,VIN_,BodyClass,Doors,DriveType,EngineCylinders,FuelTypePrimary,TransmissionStyle,Make
136163,des moines,20988,2016.0,nissan,frontier,NaN,NaN,gas,77018.0,clean,...,NaN,ia,1N6AD0EV8GN781910,pickup,NaN,4wd,6.0,Gasoline,NaN,nissan
140202,omaha / council bluffs,20988,2016.0,nissan,frontier,NaN,NaN,gas,77018.0,clean,...,NaN,ia,1N6AD0EV8GN781910,pickup,NaN,4wd,6.0,Gasoline,NaN,nissan
200708,boone,44952,2015.0,ram,3500 tradesman,NaN,NaN,diesel,153445.0,clean,...,NaN,nc,3C63R3CL1FG678469,pickup,NaN,4wd,6.0,Diesel,NaN,ram
202601,charlotte,44952,2015.0,ram,3500 tradesman,NaN,NaN,diesel,153445.0,clean,...,NaN,nc,3C63R3CL1FG678469,pickup,NaN,4wd,6.0,Diesel,NaN,ram
205074,fayetteville,44952,2015.0,ram,3500 tradesman,NaN,NaN,diesel,153445.0,clean,...,NaN,nc,3C63R3CL1FG678469,pickup,NaN,4wd,6.0,Diesel,NaN,ram
205451,greensboro,44952,2015.0,ram,3500 tradesman,NaN,NaN,diesel,153445.0,clean,...,NaN,nc,3C63R3CL1FG678469,pickup,NaN,4wd,6.0,Diesel,NaN,ram
207075,hickory / lenoir,44952,2015.0,ram,3500 tradesman,NaN,NaN,diesel,153445.0,clean,...,NaN,nc,3C63R3CL1FG678469,pickup,NaN,4wd,6.0,Diesel,NaN,ram
208473,raleigh / durham / CH,44952,2015.0,ram,3500 tradesman,NaN,NaN,diesel,153445.0,clean,...,NaN,nc,3C63R3CL1FG678469,pickup,NaN,4wd,6.0,Diesel,NaN,ram
211350,wilmington,44952,2015.0,ram,3500 tradesman,NaN,NaN,diesel,153445.0,clean,...,NaN,nc,3C63R3CL1FG678469,pickup,NaN,4wd,6.0,Diesel,NaN,ram
299441,columbia,44952,2015.0,ram,3500 tradesman,NaN,NaN,diesel,153445.0,clean,...,NaN,sc,3C63R3CL1FG678469,pickup,NaN,4wd,6.0,Diesel,NaN,ram


- updated `77176` records

# Enrich cylinders

In [22]:
df_merge.loc[(df_merge['cylinders'].notna()) & (df_merge['cylinders']!='other'),'cylinders'] = df_merge['cylinders'].str.replace(' cylinders','')

In [23]:
_cylinders_lst = df_merge.cylinders.unique().tolist()
_cylinders_lst = [x for x in _cylinders_lst if type(x) == str]
_cylinders_lst

['8', '6', '4', '5', 'other', '3', '10', '12']

In [24]:
df_merge['EngineCylinders'] = pd.to_numeric(df_merge['EngineCylinders'], errors='coerce')
df_merge['EngineCylinders'] = df_merge['EngineCylinders'].fillna(0)
df_merge['EngineCylinders'] = df_merge['EngineCylinders'].astype(int).astype(str)

In [25]:
df_merge.EngineCylinders.unique().tolist()

['0', '8', '6', '4', '3', '5', '10', '12', '2', '1']

In [26]:
df_merge[(df_merge.cylinders.isna()) & df_merge.EngineCylinders.isin(_cylinders_lst)][['VIN', 'cylinders', 'EngineCylinders']]

,VIN,cylinders,EngineCylinders
32,1GCGTDE32G1341288,NaN,6
33,5TFTX4CN3EX042751,NaN,4
35,5TFEY5F15EX169621,NaN,8
40,3TMAZ5CN7GM019182,NaN,6
41,1C4GJXAN4LW180633,NaN,4
...,...,...,...
370670,55SWF8DB6LU325050,NaN,4
370671,WDC0G4JB6JV019749,NaN,4
370674,7JR102FKXLG042696,NaN,4
370675,1GYFZFR46LF088296,NaN,4


In [27]:
df_merge.loc[(df_merge.cylinders.isna()) & df_merge.EngineCylinders.isin(_cylinders_lst), 'cylinders'] = df_merge['EngineCylinders']

- 89942 data updated

# Enrich Transmission

In [28]:
_transmission_lst = df_merge.transmission.unique().tolist()
_transmission_lst = [x for x in _transmission_lst if type(x)==str]
_transmission_lst

['other', 'automatic', 'manual']

In [29]:
df_merge.TransmissionStyle.unique().tolist()

[nan,
 'automatic',
 'manual',
 'continuously variable transmission (cvt)',
 'automated manual transmission (amt)',
 'dual-clutch transmission (dct)',
 'electronic continuously variable (e-cvt)',
 'motorcycle - chain drive']

In [30]:
df_merge[(df_merge.transmission.isna()) & df_merge.TransmissionStyle.isin(_transmission_lst)][['VIN', 'transmission', 'TransmissionStyle']]

,VIN,transmission,TransmissionStyle
6891,2HJYK16318H514382,NaN,automatic
7235,2HGES16485H620325,NaN,automatic
16131,1GCPYFELXLZ326675,NaN,automatic
22844,WVWPP7AU3GW909135,NaN,automatic
25653,1G1BE5SM7H7148682,NaN,automatic
...,...,...,...
358471,2HGFB2F82CH595481,NaN,automatic
358778,5FNYF18485B006085,NaN,automatic
358781,1G8AJ55FX6Z157867,NaN,automatic
358784,1G1JD5SH2G4172857,NaN,manual


In [31]:
df_merge.loc[(df_merge.transmission.isna()) & df_merge.TransmissionStyle.isin(_transmission_lst), 'transmission'] = df_merge['TransmissionStyle']

- 303 data updated

In [32]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370678 entries, 0 to 370677
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   region             370678 non-null  object 
 1   price              370678 non-null  int64  
 2   year               369691 non-null  float64
 3   manufacturer       355450 non-null  object 
 4   model              365993 non-null  object 
 5   condition          229397 non-null  object 
 6   cylinders          309862 non-null  object 
 7   fuel               368210 non-null  object 
 8   odometer           367189 non-null  float64
 9   title_status       363906 non-null  object 
 10  transmission       368912 non-null  object 
 11  VIN                227609 non-null  object 
 12  drive              272587 non-null  object 
 13  size               105789 non-null  object 
 14  type               311723 non-null  object 
 15  paint_color        261859 non-null  object 
 16  st

In [33]:
df_merge.FuelTypePrimary.unique()

array([nan, 'Gasoline', 'Flexible Fuel Vehicle (FFV)', 'Electric',
       'Diesel', 'Liquefied Petroleum Gas (propane or LPG)',
       'Ethanol (E85)', 'Compressed Natural Gas (CNG)', 'Fuel Cell'],
      dtype=object)

In [34]:
_fuel_lst = df_merge.fuel.unique().tolist()
_fuel_lst = [x for x in _fuel_lst if type(x)==str]
_fuel_lst

['gas', 'other', 'diesel', 'hybrid', 'electric']

In [35]:
df_merge = df_merge.replace({'FuelTypePrimary': {"Compressed Natural Gas (CNG)": 'gas', 'Liquefied Petroleum Gas (propane or LPG)': 'gas', 'Gasoline': 'gas', 'Diesel':'diesel', 'Electric':'electric'}})

In [36]:
df_merge.FuelTypePrimary.unique()

array([nan, 'gas', 'Flexible Fuel Vehicle (FFV)', 'electric', 'diesel',
       'Ethanol (E85)', 'Fuel Cell'], dtype=object)

In [37]:
df_merge[(df_merge.fuel.isna()) & df_merge.FuelTypePrimary.isin(_fuel_lst)][['VIN', 'fuel', 'FuelTypePrimary']]

,VIN,fuel,FuelTypePrimary
218,3GCUKREC2FG257362,NaN,gas
219,3GCUKREC8EG123888,NaN,gas
306,1J4RS4GG8BC644367,NaN,gas
389,3GCUKREC3GG310264,NaN,gas
517,1GC1CVCG6CF139696,NaN,gas
...,...,...,...
365873,5TDDK3EH1AS036421,NaN,gas
365998,WA1DGAFP7DA024412,NaN,gas
366128,JN8AZ18W89W124427,NaN,gas
366129,1FMYU93Z17KA66024,NaN,gas


In [38]:
df_merge.loc[(df_merge.fuel.isna()) & df_merge.FuelTypePrimary.isin(_fuel_lst), 'fuel'] = df_merge['FuelTypePrimary']

- 2109 rows updated

# `Manufacturer` Enrich

In [39]:
_manufacturer_lst = df_merge.manufacturer.unique()
_manufacturer_lst

array([nan, 'gmc', 'chevrolet', 'toyota', 'ford', 'jeep', 'nissan', 'ram',
       'mazda', 'cadillac', 'honda', 'dodge', 'lexus', 'jaguar', 'buick',
       'chrysler', 'volvo', 'audi', 'infiniti', 'lincoln', 'alfa-romeo',
       'subaru', 'acura', 'hyundai', 'mercedes-benz', 'bmw', 'mitsubishi',
       'volkswagen', 'porsche', 'kia', 'rover', 'ferrari', 'mini',
       'pontiac', 'fiat', 'tesla', 'saturn', 'mercury', 'harley-davidson',
       'datsun', 'aston-martin', 'land rover', 'morgan'], dtype=object)

In [40]:
_manufacturer_lst = [x for x in _manufacturer_lst.tolist() if type(x)==str]

In [41]:
df_merge.Make.unique()

array([nan, 'gmc', 'chevrolet', 'toyota', 'jeep', 'ford', 'nissan', 'ram',
       'mazda', 'cadillac', 'honda', 'lexus', 'jaguar', 'buick', 'volvo',
       'audi', 'infiniti', 'lincoln', 'alfa romeo', 'acura', 'hyundai',
       'bmw', 'chrysler', 'mitsubishi', 'smart', 'kia', 'maserati',
       'porsche', 'suzuki', 'dodge', 'mercedes-benz', 'subaru', 'ferrari',
       'mini', 'pontiac', 'volkswagen', 'fiat', 'land rover', 'genesis',
       'hummer', 'isuzu', 'hino', 'international', 'mitsubishi fuso',
       'freightliner', 'blue bird', 'workhorse', 'tesla',
       'sterling truck', 'bentley', 'yamaha', 'mercury', 'oldsmobile',
       'saab', 'saturn', 'western star', 'harley-davidson', 'lamborghini',
       'kenworth', 'peterbilt', 'ic bus', 'fisker', 'greengo tek', 'gm',
       'volvo truck', 'thomas built', 'am general',
       'classic motorcycles & sidecars', 'datsun', 'lotus', 'gem',
       'rolls-royce', 'zap', 'aston martin', 'titan',
       'the vehicle production group', 'kaw

In [42]:
df_merge = df_merge.replace({'Make': {'mitsubishi fuso': 'mitsubishi', 'aston martin':'aston-martin'}})

In [43]:
df_merge[(df_merge.manufacturer.isna()) & df_merge.Make.isin(_manufacturer_lst)][['VIN', 'manufacturer', 'Make']]

,VIN,manufacturer,Make
89,JTNKARJEXGJ517925,NaN,toyota
127,JTNKARJE4GJ508198,NaN,toyota
186,1GYKNERSXJZ160440,NaN,cadillac
341,3MW5R1J08L8B21625,NaN,bmw
552,SAJBD4FX6JCY57725,NaN,jaguar
...,...,...,...
369974,JTNKARJE5GJ518108,NaN,toyota
370053,JTKKU4B42C1019855,NaN,toyota
370511,JTKKU4B48AJ058849,NaN,toyota
370629,JTKKU4B42C1019855,NaN,toyota


In [44]:
df_merge.loc[(df_merge.manufacturer.isna()) & df_merge.Make.isin(_manufacturer_lst),'manufacturer'] = df_merge['Make']

- 2885 rows updated

In [45]:
# Finally Drop columns added as merge

In [46]:
df_merge.columns

Index(['region', 'price', 'year', 'manufacturer', 'model', 'condition',
       'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'VIN',
       'drive', 'size', 'type', 'paint_color', 'state', 'VIN_', 'BodyClass',
       'Doors', 'DriveType', 'EngineCylinders', 'FuelTypePrimary',
       'TransmissionStyle', 'Make'],
      dtype='object')

In [47]:
df_merge.drop(columns=df_enrich.columns, inplace=True)

In [48]:
df_merge.to_csv('data/vehicles_v2.csv', index=False)

In [51]:
df_merge.groupby('year')['year'].count()

year
1900.0        9
1901.0        3
1902.0        1
1903.0       12
1905.0        1
          ...  
2018.0    32615
2019.0    22718
2020.0    17599
2021.0     2086
2022.0      118
Name: year, Length: 114, dtype: int64